In [5]:
#For the import of packages
import numpy as np
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt
import qiime2 as q2

data_dir = "data"
local_dir = "ASV"

In [7]:
! pwd

/home/jovyan/Assignments/w4/HelicopterBaby/ASV


Check out the .qza file

In [14]:
! qiime tools peek ../$data_dir/sequences_demux_paired.qza

UUID:        b5fec962-ca06-4df5-b043-3aa289e4d753
Type:        SampleData[PairedEndSequencesWithQuality]
Data format: SingleLanePerSamplePairedEndFastqDirFmt


In [ ]:
! qiime demux summarize \
    --i-data ../$data_dir/sequences_demux_paired.qza \
    --o-visualization ../$data_dir/demux_seqs.qzv

In [ ]:
Visualization.load(f'../{data_dir}/demux_seqs.qzv')

**Brief Summary**
